In [1]:
import pandas as pd 
import pickle


In [2]:
from scipy.stats import poisson

In [3]:
dict_table=pickle.load(open("dict_table","rb"))
df_historical_data=pd.read_csv("clean_FIFA.csv")
df_fixture=pd.read_csv("clean_fixtures.csv")

In [8]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [37]:
dict_table["GrupoA"]

,Team,Pts
0,Qatar (H),0
1,Ecuador,0
2,Senegal,0
3,Netherlands,0


### Calcular Team strenght

In [10]:
## se divide el dataframe en dos 
df_home= df_historical_data[["HomeTeam","HomeGoals","AwayGoals"]]
df_away= df_historical_data[["AwayTeam","HomeGoals","AwayGoals"]]

In [15]:
## Se renombra las columnas
df_home=df_home.rename(columns={"HomeTeam":"Team","HomeGoals":"GoalsScored","AwayGoals":"GoalsConceded"})
df_away=df_away.rename(columns={"AwayTeam":"Team","HomeGoals":"GoalsConceded","AwayGoals":"GoalsScored"})

In [17]:
## Se vuelve a unir los dataframes 
df_team_strenght=pd.concat([df_home,df_away],ignore_index=True).groupby("Team").mean()

In [57]:
df_team_strenght

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [18]:
## Distribucion de poissonn, es una distribucion discreta que describe
## el numero de eventos que ocurren en un intervalo de tiempofijo o region de oportunidad:
def predict_points(home, away):
    if home in df_team_strenght.index and away in df_team_strenght.index: ## se revisan que los dos equipos esten en la tabla 
        # goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home,'GoalsScored'] * df_team_strenght.at[away,'GoalsConceded']
        lamb_away = df_team_strenght.at[away,'GoalsScored'] * df_team_strenght.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [32]:
## Testear la funcion , England united states, Qatar, Ecuador
predict_points("England","United States")

(2.2356147635326007, 0.5922397535606193)

### Prediccion del mundial

In [33]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [58]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [71]:
dict_table

{'GrupoA':           Team  Pts
 0    Qatar (H)    0
 1      Ecuador    0
 2      Senegal    0
 3  Netherlands    0,
 'GrupoB':             Team  Pts
 0        England    0
 1           Iran    0
 2  United States    0
 3          Wales    0,
 'GrupoC':            Team  Pts
 0     Argentina    0
 1  Saudi Arabia    0
 2        Mexico    0
 3        Poland    0,
 'GrupoD':         Team  Pts
 0     France    0
 1  Australia    0
 2    Denmark    0
 3    Tunisia    0,
 'GrupoE':          Team  Pts
 0       Spain    0
 1  Costa Rica    0
 2     Germany    0
 3       Japan    0,
 'GrupoF':       Team  Pts
 0  Belgium    0
 1   Canada    0
 2  Morocco    0
 3  Croatia    0,
 'GrupoG':           Team  Pts
 0       Brazil    0
 1       Serbia    0
 2  Switzerland    0
 3     Cameroon    0,
 'GrupoH':           Team  Pts
 0     Portugal    0
 1        Ghana    0
 2      Uruguay    0
 3  South Korea    0}

In [80]:
for group in dict_table:
     print(df_fixture_group_48[df_fixture_group_48['home'].isin(["Qatar"])])

NameError: name 'Qatar' is not defined

In [41]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [42]:
dict_table["GrupoA"]

,Team,Pts
0,Qatar (H),0
1,Ecuador,0
2,Senegal,0
3,Netherlands,0


In [46]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Winners Group A,Match 49,Runners-up Group B,2022,?
49,Winners Group C,Match 50,Runners-up Group D,2022,?
50,Winners Group D,Match 52,Runners-up Group C,2022,?
51,Winners Group B,Match 51,Runners-up Group A,2022,?
52,Winners Group E,Match 53,Runners-up Group F,2022,?
53,Winners Group G,Match 54,Runners-up Group H,2022,?
54,Winners Group F,Match 55,Runners-up Group E,2022,?
55,Winners Group H,Match 56,Runners-up Group G,2022,?


In [47]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [49]:
## Quarter Final
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [50]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,?,Match 58,?,2022,?
57,?,Match 57,?,2022,?
58,?,Match 60,?,2022,?
59,?,Match 59,?,2022,?


In [51]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,?,Match 58,?,2022,?
57,?,Match 57,?,2022,?
58,?,Match 60,?,2022,?
59,?,Match 59,?,2022,?


In [53]:
## Semifinal
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,?,Match 61,?,2022,?
61,?,Match 62,?,2022,?


In [54]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,?,Match 61,?,2022,?
61,?,Match 62,?,2022,?


In [55]:
##Final
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,?,Match 64,?,2022,?


In [56]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,?,Match 64,?,2022,?
